In [1]:
import torch
from torch import nn
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from scipy.ndimage import shift


In [2]:
from timeit import default_timer as timer 
def print_train_time(start: float, end: float, device: torch.device = None):
    
    total_time = end - start
    print(f"Train time on {device}: {total_time:.3f} seconds")
    return total_time

In [3]:
import requests
from pathlib import Path 

if Path("helper_functions.py").is_file():
  print("helper_functions.py already exists, skipping download")
else:
  print("Downloading helper_functions.py")
  request = requests.get("https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/helper_functions.py")
  with open("helper_functions.py", "wb") as f:
    f.write(request.content)

helper_functions.py already exists, skipping download


In [4]:
train_df = pd.read_csv(r'D:\Kaggle\digit-recognizer\train.csv')
test_df = pd.read_csv(r'D:\Kaggle\digit-recognizer\test.csv')

In [5]:
def row_operation(row, direction):
    label = row.iloc[0]
    image = row.iloc[1:].values.reshape((28,28))
    
    if direction == 'left':
        shifted_img = shift(image, shift=[0, -1], mode='constant', cval=0)
    elif direction == 'right':
        shifted_img = shift(image, shift=[0, 1], mode='constant', cval=0)
    elif direction == 'top':
        shifted_img = shift(image, shift=[-1, 0], mode='constant', cval=0)
    elif direction == 'bottom':
        shifted_img = shift(image, shift=[1, 0], mode='constant', cval=0)
    
    
    return [label]+shifted_img.flatten().tolist()

# Apply the function row-wise for different shifts
df_left = train_df.apply(lambda row: row_operation(row, 'left'), axis=1,result_type='expand')
df_right = train_df.apply(lambda row: row_operation(row, 'right'), axis=1,result_type='expand')
df_top = train_df.apply(lambda row: row_operation(row, 'top'), axis=1,result_type='expand')
df_bottom = train_df.apply(lambda row: row_operation(row, 'bottom'), axis=1,result_type='expand')

In [6]:
df_left.columns= df_right.columns= df_top.columns= df_bottom.columns = train_df.columns

train_df = pd.concat([train_df,df_left,df_right,df_bottom,df_top],axis = 0).reset_index(drop=True)

In [7]:
X = train_df.drop(['label'],axis=1)
y = train_df['label']

In [8]:
X = X / 255.0
test_df = test_df / 255.0

In [9]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
X_train = torch.tensor(X_train.values, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.long)
X_val = torch.tensor(X_val.values, dtype=torch.float32)
y_val = torch.tensor(y_val.values, dtype=torch.long)
X_test = torch.tensor(test_df.values,dtype = torch.float32)

In [11]:
X_train.shape

torch.Size([168000, 784])

In [12]:
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)
test_dataset = TensorDataset(X_test)

In [13]:
classes = y.nunique()

In [14]:
BATCH_SIZE = 128

In [15]:
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [16]:
help(train_dataloader)

Help on DataLoader in module torch.utils.data.dataloader object:

class DataLoader(typing.Generic)
 |  DataLoader(dataset: torch.utils.data.dataset.Dataset[+T_co], batch_size: Optional[int] = 1, shuffle: Optional[bool] = None, sampler: Union[torch.utils.data.sampler.Sampler, Iterable, NoneType] = None, batch_sampler: Union[torch.utils.data.sampler.Sampler[List], Iterable[List], NoneType] = None, num_workers: int = 0, collate_fn: Optional[Callable[[List[~T]], Any]] = None, pin_memory: bool = False, drop_last: bool = False, timeout: float = 0, worker_init_fn: Optional[Callable[[int], NoneType]] = None, multiprocessing_context=None, generator=None, *, prefetch_factor: Optional[int] = None, persistent_workers: bool = False, pin_memory_device: str = '')
 |
 |  Data loader combines a dataset and a sampler, and provides an iterable over the given dataset.
 |
 |  The :class:`~torch.utils.data.DataLoader` supports both map-style and
 |  iterable-style datasets with single- or multi-process load

In [17]:
class MNIST_NN(nn.Module):
    def __init__(self,input_shape: int, hidden_units: int, output_shape: int):
        super().__init__()
        self.layer_stack = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=input_shape,out_features=hidden_units),
            nn.ReLU(),
            nn.Linear(in_features=hidden_units,out_features=hidden_units),
            nn.ReLU(),
            nn.Linear(in_features=hidden_units,out_features=output_shape),
        )

    def forward(self,X):
        return self.layer_stack(X)
        

In [18]:
from helper_functions import accuracy_fn

model = MNIST_NN(784,32,classes)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.1)

In [19]:
import torch

def train_mode(model: torch.nn.Module, data_loader: torch.utils.data.DataLoader, 
               loss_fn: torch.nn.Module, accuracy_fn, optimizer: torch.optim.Optimizer):
    model.train()
    train_loss = 0
    train_accuracy = 0
    for batch, (X, y) in enumerate(data_loader):                
        y_pred = model(X)
        loss = loss_fn(y_pred, y)
        train_loss += loss.item()
        train_accuracy += accuracy_fn(y_pred.argmax(dim=1), y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch % 400 == 0:
            print(f"Looked at {batch * len(X)}/{len(data_loader.dataset)} samples")
    train_loss /= len(data_loader)
    train_accuracy /= len(data_loader)
    return train_loss, train_accuracy

def test_mode(model: torch.nn.Module, data_loader: torch.utils.data.DataLoader, 
              loss_fn: torch.nn.Module, accuracy_fn, optimizer: torch.optim.Optimizer):
    model.eval()
    test_loss = 0
    test_accuracy = 0
    with torch.inference_mode():
        for batch, (X, y) in enumerate(data_loader):
            test_pred = model(X)
            test_loss += loss_fn(test_pred, y).item()
            test_accuracy += accuracy_fn(test_pred.argmax(dim=1), y)
            if batch % 400 == 0:
                print(f"Looked at {batch * len(X)}/{len(data_loader.dataset)} samples")
        test_loss /= len(data_loader)
        test_accuracy /= len(data_loader)
    return test_loss, test_accuracy


In [20]:
from tqdm.auto import tqdm
torch.manual_seed(42)
train_time_start_on_cpu = timer()

epochs = 20
for epoch in tqdm(range(epochs)):
    print(f"Epoch: {epoch}\n---------")
    train_loss, train_accuracy = train_mode(model,train_dataloader,loss_fn,accuracy_fn,optimizer)
    print(f"\nTrain loss: {train_loss:.5f} | Train accuracy: {train_accuracy:.5f}")
    test_loss, test_accuracy = test_mode(model,val_dataloader,loss_fn,accuracy_fn,optimizer)
    print(f"\nTrain loss: {test_loss:.5f} | Train accuracy: {test_accuracy:.5f}")

train_time_end_on_cpu = timer()
total_train_time_model = print_train_time(start=train_time_start_on_cpu, 
                                           end=train_time_end_on_cpu,
                                           device=str(next(model.parameters()).device))

  0%|          | 0/20 [00:00<?, ?it/s]

Epoch: 0
---------
Looked at 0/168000 samples
Looked at 51200/168000 samples
Looked at 102400/168000 samples
Looked at 153600/168000 samples

Train loss: 0.53249 | Train accuracy: 84.34108
Looked at 0/42000 samples

Train loss: 0.28007 | Train accuracy: 91.74820
Epoch: 1
---------
Looked at 0/168000 samples
Looked at 51200/168000 samples
Looked at 102400/168000 samples
Looked at 153600/168000 samples

Train loss: 0.21461 | Train accuracy: 93.67741
Looked at 0/42000 samples

Train loss: 0.18684 | Train accuracy: 94.43152
Epoch: 2
---------
Looked at 0/168000 samples
Looked at 51200/168000 samples
Looked at 102400/168000 samples
Looked at 153600/168000 samples

Train loss: 0.15906 | Train accuracy: 95.26609
Looked at 0/42000 samples

Train loss: 0.16856 | Train accuracy: 94.91594
Epoch: 3
---------
Looked at 0/168000 samples
Looked at 51200/168000 samples
Looked at 102400/168000 samples
Looked at 153600/168000 samples

Train loss: 0.13329 | Train accuracy: 96.02770
Looked at 0/42000 samp

In [21]:
def eval_model(model: torch.nn.Module, 
               data_loader: torch.utils.data.DataLoader, 
               loss_fn: torch.nn.Module, 
               accuracy_fn):
    loss, acc = 0, 0
    model.eval()
    with torch.inference_mode():
        for X,y in data_loader:
            y_pred = model(X)
            loss += loss_fn(y_pred,y)
            acc += accuracy_fn(y,y_pred.argmax(dim=1))
        loss /= len(data_loader)
        acc /= len(data_loader)

    return {"model_name": model.__class__.__name__, # only works when model was created with a class
            "model_loss": loss.item(),
            "model_acc": acc}

simple_results = eval_model(model=model, data_loader=val_dataloader,
    loss_fn=loss_fn, accuracy_fn=accuracy_fn
)
simple_results

{'model_name': 'MNIST_NN',
 'model_loss': 0.09371914714574814,
 'model_acc': 97.3879179331307}

In [22]:

def eval_model(model: torch.nn.Module, 
               data_loader: torch.utils.data.DataLoader):
    model.eval()
    predictions = []

    with torch.inference_mode():
        for X in data_loader:
            y_pred = model(X[0])
            predictions.append(y_pred.argmax(dim=1).numpy())

    # Concatenate all batch predictions
    predictions = np.concatenate(predictions, axis=0)
    
    return {"model_name": model.__class__.__name__,  # Only works when model was created with a class
            "predictions": predictions}


In [23]:
output = eval_model(model,test_dataloader)

In [24]:
type(output)

dict

In [25]:
submission = pd.DataFrame({
    'ImageId': range(1, len(output['predictions']) + 1),
    'Label': output['predictions']
})

submission.to_csv(r'D:\Kaggle\digit-recognizer\nn_submission.csv', index=False)

print("Predictions saved to 'random_submission.csv'")

Predictions saved to 'random_submission.csv'


In [26]:
for x in test_dataloader:
    print(x[0])
    break

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


In [27]:
output['predictions'].shape

(28000,)

In [28]:
output['predictions']

array([2, 0, 9, ..., 3, 9, 2], dtype=int64)